In [1]:
# Math and data
import numpy as np
import pandas as pd
import polars as pl
import math
# Neural network frameworks
import torch as th
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, RobertaTokenizer
# Utilities
import re
from enum import Enum
import contractions as ct
import utility as util
import json
import random
from torch.utils.tensorboard import SummaryWriter
# Plotting
import matplotlib.pyplot as plt

# Pytorch device
device = th.device("mps") if th.backends.mps.is_available() else th.device("cuda") if th.cuda.is_available() else th.device("cpu")
if device.type == "cuda":
    print(th.cuda.get_device_name(device))
else:
    print(device)

device = th.device("cpu")

mps


In [2]:
# instantiate roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# write dictionary to file
def write_dict_to_file(dictionary, file_name):
    with open(file_name, 'w') as f:
        json.dump(dictionary, f)

write_dict_to_file(tokenizer.get_vocab(), "roberta_vocab.txt")

In [3]:
# Use simple GPT2 tokenizer for counting tokens
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load dataset -> Prune dataset -> Tokenize dataset
df = util.load_dataset(util.Paths.gift, util.DatasetType.ORIGINAL)
df = util.prune(df)
util.save_dataset(df, util.Paths.gift, util.DatasetType.PRUNED)
df = util.tokenize(df, tokenizer)

# Find max token length of review text with numpy
max_review_len = np.max(list(df['reviewText'].apply(list).apply(len)))
print("\nMax token length of review text: ", max_review_len)
# Find max token length of summary with numpy
max_summary_len = np.max((list(df['summary'].apply(list).apply(len))))
print("Max token length of summary: ", max_summary_len)


Max token length of review text:  553
Max token length of summary:  27


In [4]:
# torch dataset from pandas dataframe
# defines a voacbulary of words and converts the review text to a list of indices
# beware of symbols like ., !, ? etc.
# pad the review text and summary to max_review_len and max_summary_len respectively

"""
ReviewDataset pytorch dataset interface
- expects a polars dataframe with columns reviewText, summary, overall
- expects it in the DatasetType.PRUNED format
- expects a GPT2Tokenizer
"""
class ReviewDataset(Dataset):
    def __init__(self, df: pl.DataFrame, tokenizer: GPT2Tokenizer, dataset_type = util.DatasetType.PRUNED, max_review_len = 2000, max_summary_len = 200, lower_case = True, device = "cpu"):
        self.df = util.load_dataset(util.Paths.gift, dataset_type)
        self.dataset_type = dataset_type
        self.max_review_len = max_review_len
        self.max_summary_len = max_summary_len
        self.tokenizer = tokenizer
        self.lower_case = lower_case
        self.device = device

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        review = self.df["reviewText"][idx]
        summary = self.df["summary"][idx]
        rating = th.tensor(self.df["overall"][idx])

        # Tokenize the review and summary strings
        review = self.tokenizer.encode(review, add_special_tokens = True, padding = "max_length", truncation = True, max_length=self.max_review_len, return_tensors = "pt").squeeze()
        summary = self.tokenizer.encode(summary, add_special_tokens = True, padding = "max_length", truncation = True, max_length=self.max_summary_len, return_tensors = "pt").squeeze()

        # move tensors to device
        review = review.to(self.device)
        summary = summary.to(self.device)
        rating = rating.to(self.device)
        
        return review, summary, rating
    
    def detokenize(self, x: th.Tensor):
        return self.tokenizer.decode(x, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    def batch_detokenize(self, x: th.Tensor):
        return [self.detokenize(x[i]) for i in range(len(x))]

In [5]:
# Test the dataset
# Setup
t = GPT2Tokenizer.from_pretrained("gpt2", add_bos_token=True, add_prefix_space=True, trim_offsets=True)
t.pad_token = t.eos_token
t.add_special_tokens({"bos_token": util.BOS_token})

# Create the dataset
dataset = ReviewDataset(util.Paths.gift, t, max_review_len = max_review_len, max_summary_len = max_summary_len, lower_case = False, device=device)

data_idx = 45
# print(f"Review: {dataset[data_idx][0]}")

# decode
print(f"Review: {ReviewDataset.detokenize(dataset, dataset[data_idx][0])}")
print(f"Summary: {ReviewDataset.detokenize(dataset, dataset[data_idx][1])}")
print(f"Rating: {int(dataset[data_idx][2])}")

# max length is the max index of the vocabulary
MAX_LENGTH = len(t)
print(f"MAX_LENGTH: {MAX_LENGTH}")

Review:  what is to go wrong with a gift card. as long as it enters into a person's account as a credit it is just what you paid for.
Summary:  gift card
Rating: 1
MAX_LENGTH: 50258


In [6]:
"""
Model
"""

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return th.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = nn.functional.softmax(
            self.attn(th.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = th.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = th.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = nn.functional.relu(output)
        output, hidden = self.gru(output, hidden)

        output = nn.functional.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return th.zeros(1, 1, self.hidden_size, device=device)

In [7]:
# Test the model with a single forward pass
hidden_size = 256
encoder = EncoderRNN(MAX_LENGTH, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, MAX_LENGTH, dropout_p=0.1).to(device)

# dl = DataLoader(dataset)
# dl_it = iter(dl)

# Take input from the dataset
input_tensor, target_tensor, rating_tensor = dataset[data_idx]
# print(input_tensor.get_device())

# Create the encoder hidden state
encoder_hidden = encoder.initHidden()

# Initialise the encoder output
encoder_outputs = th.zeros(MAX_LENGTH, encoder.hidden_size, device=device)

# Run the encoder
for token in input_tensor:
    # print(token)
    encoder_output, encoder_hidden = encoder(token, encoder_hidden)
    encoder_outputs[token] = encoder_output[0, 0]

bos = th.tensor(t.bos_token_id).to(device)

# Create the decoder input
decoder_input = th.tensor([bos], device=device, dtype=th.long)
# Create the decoder output
decoder_output_sequence = th.zeros(max_summary_len, device=device, dtype=th.long)

# Create the decoder hidden state
decoder_hidden = encoder_hidden

# Run the decoder
for i, target in enumerate(target_tensor):
    decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
    topv, topi = decoder_output.topk(1)
    decoder_input = topi.squeeze().detach() # detach from history as input
    
    # Append the output
    decoder_output_sequence[i] = decoder_input

    if decoder_input.item() == t.eos_token_id:
        print(f"EOS token found at {i}th iteration")
        break

# Print the output before detokenization
print(f"Output:\n{decoder_output_sequence}\n")

# Print the detokenized output
print(f"Detokenized output:\n{ReviewDataset.detokenize(dataset, decoder_output_sequence)}\n")

Output:
tensor([ 7552,  6482, 17535, 17535, 29983, 43434, 17535, 29983, 30825, 39823,
        48076, 28777,  2311, 30825,  2311, 30825, 11442, 19782, 21797, 11623,
        32379, 13923, 19792, 10766, 18720, 12654, 35905])

Detokenized output:
 extension shift accessed accessedpurpose6000 accessedpurpose Xavier Jinping FTA obedience Sen Xavier Sen Xavier110190activity125 Crab witnessed zoom Deep Codñ infuri



In [8]:
""" 
Training
"""
n_epochs = 1
batch_size = 8
learning_rate = 0.5
teacher_forcing_ratio = 0.5

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", add_bos_token=True, add_prefix_space=True, trim_offsets=True)
criterion = nn.CrossEntropyLoss() # TODO: Check without the ignore_index
#criterion = nn.NLLLoss()

encoder = EncoderRNN(MAX_LENGTH, hidden_size).to(device).train()
decoder = AttnDecoderRNN(hidden_size, MAX_LENGTH, dropout_p=0.1).to(device).train()

encoder_optimizer = th.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = th.optim.Adam(decoder.parameters(), lr=learning_rate)

# Create the dataset
dataset = ReviewDataset(util.Paths.gift, t, max_review_len = max_review_len, max_summary_len = max_summary_len, lower_case = False, device=device)

# Calculate the number of elements in each bucket
split_ratios = [0.7, 0.2, 0.1]
num_reviwes = len(dataset)
n_train_dataset = math.floor(num_reviwes * split_ratios[0])
n_validate_dataset = math.floor(num_reviwes * split_ratios[1])
n_test_dataset = num_reviwes - n_train_dataset - n_validate_dataset
assert n_train_dataset + n_validate_dataset + n_test_dataset == num_reviwes
#print(len(dataset))
#print(n_train_dataset +n_validate_dataset + n_test_dataset)
train_dataset, val_dataset, test_dataset = th.utils.data.random_split(dataset, [n_train_dataset, n_validate_dataset, n_test_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

batch_sequence = next(iter(train_loader))

# Readying the writer
writer = SummaryWriter()

# Training loop
def train(learning_rate, n_epochs, train_loader, valid_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    for epoch in range(n_epochs):
        for batch_idx, (train_review_batch, train_summary_batch, train_rating_batch) in enumerate(train_loader):
            batch_loss = 0
            words_in_batch = 0

            # We calcualte the loss and backpropagate every batch
            # Reset the gradients
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()

            for review, summary, rating in zip(train_review_batch, train_summary_batch, train_rating_batch):
                # Create the encoder hidden state
                encoder_hidden = encoder.initHidden() # Can be understood as the context vector

                # Initialise the encoder output's "feature space"
                encoder_outputs = th.zeros(MAX_LENGTH, encoder.hidden_size, device=device)

                # Run the encoder
                for token in review:
                    encoder_output, encoder_hidden = encoder(token, encoder_hidden)
                    encoder_outputs[token] = encoder_output[0, 0]

                # Create the decoder input, the beginning of the sequence, starting with the BOS (Beginning Of String) token
                decoder_input = th.tensor([bos], device=device, dtype=th.long)

                # Initialize the decoder output
                decoder_output_sequence = th.zeros(max_summary_len, device=device, dtype=th.float)

                # Propagate the decoder hidden state
                decoder_hidden = encoder_hidden

                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

                if use_teacher_forcing:
                    # Teacher forcing: Feed the target as the next input
                    for target_index, target in enumerate(summary):
                        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                        batch_loss += criterion(decoder_output, target)
                        decoder_input = target # Teacher forcing
                else:
                    # Run the decoder
                    for target_index, target in enumerate(summary):
                        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                        topv, topi = decoder_output.topk(1)
                        decoder_input = topi.squeeze().detach() # detach from history as input
                        
                        # Append the output
                        decoder_output_sequence[target_index] = decoder_input

                        if decoder_input.item() == t.eos_token_id:
                            #print(f"EOS token found at {target_index}th iteration")
                            # Print the detokenized output
                            # Print the target sequence
                            # print(f"Target sequence:\n{dataset.detokenize(summary)}")
                            # print(f"Detokenized output:\n{dataset.detokenize(decoder_output_sequence)}\n")
                            break

                        words_in_batch += 1
                        # Calculate the loss
                        batch_loss += criterion(decoder_output.squeeze(), target)
                print(f"Target sequence:\n{dataset.detokenize(summary)}")
                print(f"Detokenized output:\n{dataset.detokenize(decoder_output_sequence)}\n")
            # Backpropagate the loss
            batch_loss.backward()

            # Update the weights
            encoder_optimizer.step()
            decoder_optimizer.step()

            # Print the loss
            writer.add_scalar("Loss/train", batch_loss/words_in_batch, epoch * len(train_loader) + batch_idx)
            print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {batch_loss/words_in_batch}")
                
train(learning_rate, n_epochs, train_loader, valid_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

Target sequence:
 subway gift cards
Detokenized output:
 Ram Cooperation Tad Tracks cape claimantsMenuMenuMenu fiber Cosmicarter Cosmic moving cinnamon Literary Cosmic labs modifyMenuMenuMenuMenu fiberarthy cinnamon Aph



UnboundLocalError: local variable 'loss' referenced before assignment

In [ ]:
# Test the model
def test(test_loader, encoder, decoder):
    for batch_idx, (test_review_batch, test_summary_batch, test_rating_batch) in enumerate(test_loader):
        for review, summary, rating in zip(test_review_batch, test_summary_batch, test_rating_batch):
            # Create the encoder hidden state
            encoder_hidden = encoder.initHidden() # Can be understood as the context vector

            # Initialise the encoder output's "feature space"
            encoder_outputs = th.zeros(MAX_LENGTH, encoder.hidden_size, device=device)

            # Run the encoder
            for token in review:
                encoder_output, encoder_hidden = encoder(token, encoder_hidden)
                encoder_outputs[token] = encoder_output[0, 0]

            # Create the decoder input, the beginning of the sequence, starting with the BOS (Beginning Of String) token
            decoder_input = th.tensor([bos], device=device, dtype=th.long)

            # Create the decoder output
            decoder_output_sequence = th.zeros(max_summary_len, device=device, dtype=th.float)

            # Propagate the decoder hidden state
            decoder_hidden = encoder_hidden

            # Run the decoder
            for target_index, target in enumerate(summary):
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach() # detach from history as input
                
                # Append the output
                decoder_output_sequence[target_index] = decoder_input

                if decoder_input.item() == t.eos_token_id:
                    print(f"EOS token found at {target_index}th iteration")
                    break

            # Print the output before detokenization
            print(f"Output:\n{decoder_output_sequence}\n")

            # Print the detokenized output
            print(f"Detokenized output:\n{ReviewDataset.detokenize(dataset, decoder_output_sequence)}\n")

test(test_loader, encoder, decoder)